# NodeJS MySQL AES_ENCRYPT / AES_DECRYPT
#### A NodeJS implementation of MySQL's AES_ENCRYPT/AES_DECRYPT functions with built-in MD5 key hashing

### Background

The MyCipher class below reproduces the functionality of the SQL AES encryption functions and includes built-in MD5 key hashing in order to mirror the recommended usage, as in the following SQL queries:
```sql
SELECT HEX( AES_ENCRYPT('secret value', MD5('encryption key') ) );
SELECT AES_DECRYPT(<ciphertext>, MD5('encryption key') );
```
Equuivalent functions using MyCipher:
```javascript
var cipher = new MyCipher('encryption key');
cipher.encrypt('secret value');
cipher.decrypt(<ciphertext>);
```

##### Note
Since ECB mode doesn't use IVs, it is important to salt the keys passed to MyCipher's constructor using a salt that will be know during lookup. e.g. A user ID could be prepended to the encryption key to ensure that each key is unique.

In [1]:
var crypto = require('crypto');
var md5 = require('md5');

var MyCipher = function(key){
    this.key = this.setKey(key);
    this.cipher = crypto.createCipheriv('aes-128-ecb', this.key, '');
}
MyCipher.prototype._pad = function(s){
    var pad_value = 16 - (s.length % 16);
    return s + String.fromCharCode(pad_value).repeat(pad_value);
}
MyCipher.prototype.setKey = function(key){
    key = md5(key);
    var final_key = new Uint8Array(16); //Start with 16-byte block of null characters.
    for (var i=0; i<key.length; i++){   //Iterate over all characters in our key.
        final_key[i%16] = final_key[i%16] ^ key.charCodeAt(i); // XOR the characters together.
    }
    // Convert to byte string.
    this.key = String.fromCharCode.apply(null, new Uint8Array(final_key));
    return this.key;
}
MyCipher.prototype.getKey = function(){
    return new Buffer(this.key).toString('hex');
};
MyCipher.prototype.encrypt = function(plaintext){
    return this.cipher.update(this._pad(plaintext),'utf8','hex');
};
MyCipher.prototype.decrypt = function(ciphertext){
    var cipher = crypto.createDecipheriv('aes-128-ecb', this.key, "");
    return cipher.update(ciphertext, 'hex', 'utf8') + cipher.final('utf8');
};
console.log();

### Example usage

In [2]:
var c = new MyCipher('D476D59919BC');
var ciphertext = c.encrypt('foo');
var buf = new Buffer(ciphertext,'hex');

console.log("encryption key:",c.getKey());
console.log("encrypted:",ciphertext);
console.log("decrypted:",c.decrypt(ciphertext));

encryption key: 5d5b0005030f0108500a01540c570705
encrypted: 961d72fbde896818a97d6810c449fb90
decrypted: foo


### Verify with MySQL
```mysql
select hex(aes_encrypt('foo', MD5('D476D59919BC')));
```
```mysql
select aes_decrypt(X'961D72FBDE896818A97D6810C449FB90',MD5('D476D59919BC'));
```


In [3]:
var mysql = require('mysql');

// no need to configure host since we don't actually query any tables
var connection = mysql.createConnection({}); 
connection.connect();

var myCiphertext = new MyCipher('D476D59919BC').encrypt('foo');
var encQuery = "select hex(aes_encrypt('foo', MD5('D476D59919BC'))) as ciphertext;";
connection.query(encQuery, function(err, rows, fields) {
    if (err) throw err;
    console.log('Got: "'+rows[0].ciphertext+'", Expected: "'+myCiphertext+'"');
    // now check the other direction
    var decQuery = "select aes_decrypt(X'"+rows[0].ciphertext+"',MD5('D476D59919BC')) as res;";
    connection.query(decQuery, function(err, rows, fields) {
        if (err) throw err;
        console.log('Got: "'+rows[0].res+'", Expected: "foo"');
        connection.end();
    });
});
console.log();


Got: "961D72FBDE896818A97D6810C449FB90", Expected: "961d72fbde896818a97d6810c449fb90"
Got: "foo", Expected: "foo"
